## Практическое задание к уроку 6 по теме "Классификация текста. Анализ тональности текста".


Взять ноутбук colab_text_classification_part1.ipynb который разбирали на занятии и добавить пункты которые мы пропустили

Посмотрите на токены если будут мусорные добавьте их в стоп слова и обучите
заново
2. Проверьте изменилось ли качество при лемматизации/и без неё
3. Замените все токены которые принадлежат сущностям на их тег. Проверьте изменилось ли качество после этого

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MaxAbsScaler
import spacy
from tqdm import tqdm

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
RANDOM_STATE = 42
NUM_THREADS = 12

Загрузка данных

In [4]:
!wget -O imdb.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vrQ5czMHoO3pEnmofFskymXMkq_u1dPc"
!unzip imdb.zip
!pip -q install eli5
!pip -q install spacy
!python -m spacy download en

Archive:  imdb.zip
  inflating: test.tsv                
  inflating: train.tsv               
     |████████████████████████████████| 216 kB 14.0 MB/s 
     |████████████████████████████████| 133 kB 76.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 requires jinja2<=3.0.0, but you have jinja2 3.1.2 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-17 10:50:47.169838: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looki

Подготовка данных

In [5]:
train_df = pd.read_csv("train.tsv", delimiter="\t")
test_df = pd.read_csv("test.tsv", delimiter="\t")

print('Train size = {}'.format(len(train_df)))
print('Test size = {}'.format(len(test_df)))

Train size = 25000
Test size = 25000


Посмотрите глазами на тексты? Какие есть зацепки, как определить, что это за сентимент?

Самое простое, как всегда - найти ключевые слова.

In [7]:
#@title Начинаем классифицировать! { vertical-output: true, display-mode: "form" }
positive_words = 'love', 'great', 'best', 'wonderful' #@param {type:"raw"}
negative_words = 'worst', 'awful', 'boring', 'crap' #@param {type:"raw"}

positives_count = test_df.review.apply(lambda text: sum(word in text for word in positive_words))
negatives_count = test_df.review.apply(lambda text: sum(word in text for word in negative_words))
is_positive = positives_count > negatives_count
correct_count = (is_positive == test_df.is_positive).values.sum()

accuracy = correct_count / len(test_df)

print('Test accuracy = {:.2%}'.format(accuracy))
if accuracy > 0.71:
    from IPython.display import Image, display
    display(Image('https://s3.amazonaws.com/achgen360/t/rmmoZsub.png', width=500))

Test accuracy = 67.49%


**Задание** Придумайте хорошие ключевые слова или фразы и наберите хотя бы 71% точности на тесте (и не забудьте посмотреть на код классификации!)

In [8]:
train_df.head()

,is_positive,review
0,0,"Dreamgirls, despite its fistful of Tony wins i..."
1,0,This show comes up with interesting locations ...
2,1,I simply love this movie. I also love the Ramo...
3,0,Spoilers ahead if you want to call them that.....
4,1,My all-time favorite movie! I have seen many m...


Предобработка без лематизации

In [9]:
def removeApostrophe(review):
    phrase = re.sub(r"won't", "will not", review)
    phrase = re.sub(r"can\'t", "can not", review)
    phrase = re.sub(r"n\'t", " not", review)
    phrase = re.sub(r"\'re", " are", review)
    phrase = re.sub(r"\'s", " is", review)
    phrase = re.sub(r"\'d", " would", review)
    phrase = re.sub(r"\'ll", " will", review)
    phrase = re.sub(r"\'t", " not", review)
    phrase = re.sub(r"\'ve", " have", review)
    phrase = re.sub(r"\'m", " am", review)
    return phrase

def removeApostropheFixed(review):
    phrase = re.sub(r"won't", "willnot", review)
    phrase = re.sub(r"can\'t", "cannot", review)
    phrase = re.sub(r"\snot", "not", review)
    phrase = re.sub(r"n\'t", "not", review)
    phrase = re.sub(r"\'re", " are", review)
    phrase = re.sub(r"\'s", " is", review)
    phrase = re.sub(r"\'d", " would", review)
    phrase = re.sub(r"\'ll", " will", review)
    phrase = re.sub(r"\'t", " not", review)
    phrase = re.sub(r"\'ve", " have", review)
    phrase = re.sub(r"\'m", " am", review)
    return phrase

def removeSpecialChars(review):
     return re.sub('[^a-zA-Z]', ' ', review)
    
def doCleaningFixed(review):
    review = removeApostropheFixed(review)
    review = removeSpecialChars(review) 
    review = re.sub('br', " ", review) 
    review = review.lower()  
    review = " ".join(word for word in review.split() if word.isalpha())
    review = review.split() #Tokenization
    # lmtzr = WordNetLemmatizer()
    # review = [lmtzr.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = " ".join(review)    
    return review

def doTextCleaning(review):
    review = removeApostrophe(review)
    review = removeSpecialChars(review) 
    review = review.lower()  
    review = review.split() #Tokenization
    lmtzr = WordNetLemmatizer()
    review = [lmtzr.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = " ".join(review)    
    return review

In [10]:
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm

In [11]:
%%time
N = 1000
with Pool(NUM_THREADS) as p:
    train_df['textNew'] = list(tqdm(p.imap(doCleaningFixed, train_df['review']), total=len(train_df)))
    test_df['textNew'] = list(tqdm(p.imap(doCleaningFixed, test_df['review']), total=len(test_df)))

<timed exec>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/25000 [00:00<?, ?it/s]

<timed exec>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/25000 [00:00<?, ?it/s]

CPU times: user 5.22 s, sys: 1.48 s, total: 6.69 s
Wall time: 19.1 s


In [12]:
print(train_df['review'].iloc[3])
print(train_df['textNew'].iloc[3])

print(test_df['review'].iloc[3])
print(test_df['textNew'].iloc[3])


Spoilers ahead if you want to call them that...<br /><br />I would almost recommend this film just so people can truly see a 1/10. Where to begin, we'll start from the top...<br /><br />THE STORY: Don't believe the premise - the movie has nothing to do with abandoned cars, and people finially understanding what the mysterious happenings are. It's a draub, basic, go to cabin movie with no intensity or "effort".<br /><br />THE SCREENPLAY: I usually give credit to indie screenwriters, it's hard work when you are starting out...but this is crap. The story is flat - it leaves you emotionless the entire movie. The dialogue is extremely weak and predictable boasting lines of "Woah, you totally freaked me out" and "I was wondering if you'd uh...if you'd like to..uh, would you come to the cabin with me?". It makes me want to rip out all my hair, one strand at a time and feed it to myself.<br /><br />THE CHARACTERS: HOLY CRAP!!!! Some have described the characters as flat, I want to take it one 

In [13]:
#@title Начинаем классифицировать! { vertical-output: true, display-mode: "form" }
positive_words = 'excellent', 'great', 'loved', 'the best' #@param {type:"raw"}
negative_words = 'the worst', 'awful', 'bad', ' terrible' #@param {type:"raw"}

positives_count = test_df.review.apply(lambda text: sum(word in text for word in positive_words))
negatives_count = test_df.review.apply(lambda text: sum(word in text for word in negative_words))
is_positive = positives_count > negatives_count
correct_count = (is_positive == test_df.is_positive).values.sum()

accuracy = correct_count / len(test_df)

print('Test accuracy = {:.2%}'.format(accuracy))
if accuracy > 0.71:
    from IPython.display import Image, display
    display(Image('https://s3.amazonaws.com/achgen360/t/rmmoZsub.png', width=500))

Test accuracy = 67.29%


Построим линейную модель. Она будет учиться строить разделяющую гиперплоскость в пространстве bow-векторов. Возьмём униграммы и биграммы слов и не
забудем отмасштабировать данные перед логистической регрессией

In [14]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
scaler = MaxAbsScaler()
classifier = LogisticRegression()

model = make_pipeline(vectorizer, scaler, classifier)
model.fit(train_df['textNew'], train_df['is_positive']);

In [16]:
# функцию для оценки моделей
def eval_model(model):
    pred = model.predict(test_df['textNew'])
    return accuracy_score(test_df['is_positive'], pred)

Будем записывать все результаты в словарь для последующей
визуализации:

In [17]:
results = {}

In [18]:
results['Preprocessing'] = eval_model(model)
print(results['Preprocessing'])

0.90412


С достаточно просто предобработкой получился хороший скор.

Хочется как-то посмотреть, что заинтересовало классификатор. К счастью, сделать это совсем просто:

In [19]:
import eli5
eli5.show_weights(classifier, vec=vectorizer, top=40)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+2.325,great
+2.035,the best
+1.971,excellent
+1.764,wonderful
+1.307,loved
+1.286,well
+1.261,is great
+1.222,perfect
+1.104,definitely worth
+1.080,enjoyed


Посмотрим на конкретные примеры его работы:

In [20]:
print('Positive' if test_df['is_positive'].iloc[1] else 'Negative')
eli5.show_prediction(classifier, test_df['textNew'].iloc[1], vec=vectorizer, 
                     targets=['positive'], target_names=['negative', 'positive'])

Positive


In [21]:
print('Positive' if test_df['is_positive'].iloc[6] else 'Negative')
eli5.show_prediction(classifier, test_df['textNew'].iloc[6], vec=vectorizer, 
                     targets=['positive'], target_names=['negative', 'positive'])

Negative


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Посмотрим на примеры неправильной классификации, наконец:

In [22]:
preds = model.predict(test_df['textNew'])
incorrect_pred_index = np.random.choice(np.where(preds != test_df['is_positive'])[0])

eli5.show_prediction(classifier, test_df['textNew'].iloc[incorrect_pred_index],
                     vec=vectorizer, targets=['positive'], target_names=['negative', 'positive'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Мусорные токены 

Для обработки "лишних" токенов будем использовать аргумент
векторайзера max_df, который будет фильтровать наиболее
часто встречающиеся токены

In [23]:
vectorizer = CountVectorizer(ngram_range=(1, 2), max_df=0.4)
model = make_pipeline(vectorizer, scaler, classifier)
model.fit(train_df['textNew'], train_df['is_positive']);

In [24]:
results['Preprocessing + exclude stopwords'] = eval_model(model)
print(results['Preprocessing + exclude stopwords'])

0.90488


Результат немного улучшился

Лемматизация

Для лемматизации и замены токенов на сущности в следующем
задании воспользуемся библиотекой spacy. 

In [25]:
nlp = spacy.load('en_core_web_sm', disable=['parser'])

In [26]:
docs = nlp.pipe(train_df['textNew'])

In [27]:
lemmatized = []
for doc in tqdm(docs, total=len(train_df), position=0):
     lemmatized.append(' '.join(token.lemma_ for token in doc))

<ipython-input-27-58cea09413d2>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(docs, total=len(train_df), position=0):


  0%|          | 0/25000 [00:00<?, ?it/s]

In [28]:
train_df['review_lemmatization'] = lemmatized

In [29]:
model.fit(train_df['review_lemmatization'], train_df['is_positive'])
results['Preprocessing + lemmatization'] = eval_model(model)
print(results['Preprocessing + lemmatization'])

0.88968


Уже результат значительно хуже. Проверим
ещё результат работы, где векторайзер не фильтрует слова по
частоте. Вполне вероятно, что после лемматизации часть важных
слов отфильтровалась, т.к. слова объединились и превысили порог.

In [30]:
vectorizer.max_df = 1.0

In [31]:
model.fit(train_df['review_lemmatization'], train_df['is_positive'])
results['Preprocessing + stopwords + lemmatization'] = eval_model(model)
print(results['Preprocessing + stopwords + lemmatization'])

0.89008


Без фильтрации токенов результат еще хуже

Замена токенов на сущности

In [32]:
docs = nlp.pipe(train_df['textNew'])

In [33]:
replaced = []
for doc in tqdm(docs, total=len(train_df), position=0):
    spam = []
    for token in doc:
        spam.append(token.ent_type_ if token.ent_type else token.text)
    replaced.append(' '.join(spam))

<ipython-input-33-c900d27018cc>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(docs, total=len(train_df), position=0):


  0%|          | 0/25000 [00:00<?, ?it/s]

In [35]:
train_df['review_replace'] = replaced

Так же проверим оба варианта: с фильтрацией
слов и без неё. Потому что сущности могут оказаться
слишком частыми токенами и будут отфильтрованы.

In [36]:
model.fit(train_df['review_replace'], train_df['is_positive'])
results['Preprocessing + replace entities'] = eval_model(model)
print(results['Preprocessing + replace entities'])

0.90432


In [37]:
vectorizer.max_df = 0.4

In [38]:
model.fit(train_df['review_replace'], train_df['is_positive'])
results['Preprocessing + stopwords + entities'] = eval_model(model)
print(results['Preprocessing + stopwords + entities'])

0.905


Выведем всю таблицу

In [39]:
pd.DataFrame(results.values(), index=results.keys(), 
             columns=['accuracy']).sort_values('accuracy', ascending=False)

,accuracy
Preprocessing + stopwords + entities,0.90500
Preprocessing + exclude stopwords,0.90488
Preprocessing + replace entities,0.90432
Preprocessing,0.90412
Preprocessing + stopwords + lemmatization,0.89008
Preprocessing + lemmatization,0.88968


Лучший результат показала предобработка без лемматизации с заменой токенов на тэги сущностей и исключением часто встречающихся токенов.
